# 模拟删除

In [1]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = load_model("textbook")
ecoli_model = load_model("iJO1366")

## 敲除单个基因和反应

在分析代谢模型时，一个经常被问到的问题是，如果某种反应不允许有任何通量，会发生什么。这可以通过以下方式使用 cobrapy 进行测试

In [2]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x2551f601bd0>
pfk knocked out:  <Solution 0.704 at 0x2551f603b50>


在评估遗传的操作策略时，更有趣的是检查如果给定的基因被敲除会发生什么，因为这样做在冗余的情况下不会影响任何反应，或者如果基因参与多个反应，则会影响更多的反应。

In [3]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x2551f60ec90>
pfkA knocked out:  <Solution 0.874 at 0x2551f60e590>
pfkB knocked out:  <Solution 0.704 at 0x2551f60e0d0>


## 单次删除

在模型上执行所有单基因删除

In [4]:
deletion_results = single_gene_deletion(cobra_model)

这些也可以只对一部分基因进行

In [5]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

,ids,growth,status
0,{b1241},0.873922,optimal
1,{b1478},0.873922,optimal
2,{b0727},0.858307,optimal
3,{b2587},0.873922,optimal
4,{b1849},0.873922,optimal
5,{b0474},0.873922,optimal
6,{b2296},0.873922,optimal
7,{b0116},0.782351,optimal
8,{b0351},0.873922,optimal
9,{b3734},0.374230,optimal


这也可用于反应

In [6]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

,ids,growth,status
0,{ACONTb},-1.376403e-15,optimal
1,{ENO},8.053080e-16,optimal
2,{ACALD},8.739215e-01,optimal
3,{CYTBD},2.116629e-01,optimal
4,{ALCD2x},8.739215e-01,optimal
5,{CS},-7.494929e-15,optimal
6,{D_LACt2},8.739215e-01,optimal
7,{ACKr},8.739215e-01,optimal
8,{ETOHt2r},8.739215e-01,optimal
9,{Biomass_Ecoli_core},0.000000e+00,optimal


## 双重删除

双重删除以类似的方式运行。

In [7]:
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:]).round(4)

,ids,growth,status
0,"{b2935, b2465}",0.0000,optimal
1,"{b0008, b2935}",0.8739,optimal
2,{b2464},0.8739,optimal
3,"{b3919, b2465}",0.7040,optimal
4,{b0008},0.8739,optimal
5,"{b0008, b2464}",0.8648,optimal
6,"{b2935, b2464}",0.8739,optimal
7,{b2935},0.8739,optimal
8,"{b2935, b3919}",0.7040,optimal
9,"{b3919, b2464}",0.7040,optimal


默认情况下，双重删除功能将自动使用多重处理，如果可用，则将任务拆分为最多 4 个内核。也可以手动指定内核数。设置使用单个内核将禁用多处理库，这通常有助于调试。

In [8]:
start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=2)
t1 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 2 cores" % t1)

start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=1)
t2 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 1 core" % t2)

print("Speedup of %.2fx" % (t2 / t1))

Double gene deletions for 200 genes completed in 5.06 sec with 2 cores
Double gene deletions for 200 genes completed in 2.85 sec with 1 core
Speedup of 0.56x


也可以对反应运行双重删除。

In [9]:
double_reaction_deletion(
    cobra_model, cobra_model.reactions[2:7]).round(4)

,ids,growth,status
0,"{ACKr, ACt2r}",0.8739,optimal
1,"{ACONTb, ACt2r}",-0.0000,optimal
2,"{ADK1, ACt2r}",0.8739,optimal
3,{ACKr},0.8739,optimal
4,"{ACONTa, ACKr}",0.0000,optimal
5,"{ACONTb, ACONTa}",-0.0000,optimal
6,"{ADK1, ACONTa}",-0.0000,optimal
7,{ACONTb},-0.0000,optimal
8,"{ACONTb, ACKr}",-0.0000,optimal
9,"{ADK1, ACKr}",0.8739,optimal


## 访问单个删除结果
请注意，删除的索引是 python set 对象。这是适当的类型，因为删除的顺序无关紧要。删除反应 1 和反应 2 与删除反应 2 和反应 1 的效果相同。

为了更轻松地访问结果，COBRAPpy 删除函数返回的所有 DataFrame 都有一个 knockout 索引器，使访问变得更加简单。索引器中的每个条目都被视为单个删除条目。因此，您需要传递双重删除的集合。

In [10]:
single = single_reaction_deletion(cobra_model)
double = double_reaction_deletion(cobra_model)

print(single.knockout["ATPM"])
print(double.knockout[{"ATPM", "TKT1"}])

       ids    growth   status
92  {ATPM}  0.916647  optimal
             ids   growth   status
19  {ATPM, TKT1}  0.90584  optimal


这可用于一次获得多个删除，也适用于反应或基因对象（取决于您删除的内容）。

In [11]:
atpm = cobra_model.reactions.ATPM
tkt1 = cobra_model.reactions.TKT1
pfk = cobra_model.reactions.PFK

print(single.knockout[atpm, tkt1, pfk])
print(double.knockout[{atpm, tkt1}, {atpm, pfk}, {atpm}])


       ids    growth   status
60  {TKT1}  0.864759  optimal
81   {PFK}  0.704037  optimal
92  {ATPM}  0.916647  optimal
               ids    growth   status
19    {ATPM, TKT1}  0.905840  optimal
1136   {PFK, ATPM}  0.704037  optimal
3996        {ATPM}  0.916647  optimal
